# Analyze runtimes

Analyze the runtimes of the injections we did

## Preamble

In [339]:
import psutil
p = psutil.Process()
p.cpu_affinity([0])
import os
import json
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np 
import matplotlib.pyplot as plt
from IPython.display import Latex

params = {"axes.grid": True,
        "text.usetex" : True,
        "font.family" : "serif",
        "ytick.color" : "black",
        "xtick.color" : "black",
        "axes.labelcolor" : "black",
        "axes.edgecolor" : "black",
        "font.serif" : ["Computer Modern Serif"],
        "xtick.labelsize": 16,
        "ytick.labelsize": 16,
        "axes.labelsize": 16,
        "legend.fontsize": 16,
        "legend.title_fontsize": 16,
        "figure.titlesize": 16}

plt.rcParams.update(params)
import copy

# from utils_compare_runs import paths_dict

In [340]:
outdirs_injections = {"TF2": "../injections/outdir_TF2/",
                      "NRTv2": "../injections/outdir_NRTv2/"}

jim_root_path = "/home/thibeau.wouters/TurboPE-BNS/real_events/"
bilby_root_path = "/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/"
paths_dict = {"GW170817_TaylorF2": {"jim": jim_root_path + "GW170817_TaylorF2/outdir_snellius/",
                    "bilby": bilby_root_path + "GW170817_TF2_with_tukey_fix_result.json"},
              
              "GW170817_NRTidalv2": {"jim": jim_root_path + "GW170817_NRTidalv2/outdir_snellius/",
                                     "bilby": bilby_root_path + "GW170817_IMRPhenomD_NRTidalv2_result.json",
                    },
              
              "GW190425_TaylorF2": {"jim": jim_root_path + "GW190425_TaylorF2/outdir_snellius/",
                                    "bilby": bilby_root_path + "GW190425_TF2_with_tukey_fix_result.json",
                    },
              
              "GW190425_NRTidalv2": {"jim": jim_root_path + "GW190425_NRTidalv2/outdir_snellius/",
                                     "bilby": bilby_root_path + "GW190425_IMRPhenomD_NRTidalv2_result.json",
                    },
}

### Old version
# jim_paths = {"GW170817_TaylorF2": "../real_events/GW170817_TaylorF2/outdir/",
#              "GW170817_NRTidalv2": "../real_events/GW170817_NRTidalv2/outdir/",
#              "GW190425_TaylorF2": "../real_events/GW190425_TaylorF2/outdir/",
#              "GW190425_NRTidalv2": "../real_events/GW190425_NRTidalv2/outdir/"}

# bilby_paths = {"GW170817_TaylorF2": "/home/thibeau.wouters/jim_pbilby_samples/GW170817/GW170817-TF2_rejection_sampling_result.json",
#                "GW170817_NRTidalv2": "/home/thibeau.wouters/jim_pbilby_samples/GW170817/GW170817_IMRDNRTv2_older_bilby_result.json",
#                "GW190425_TaylorF2": "/home/thibeau.wouters/jim_pbilby_samples/GW190425/GW190425-TF2_result.json",
#                "GW190425_NRTidalv2": "/home/thibeau.wouters/jim_pbilby_samples/GW190425/GW190425_IMRDNRTv2_older_bilby_result.json"}

# run_names = list(jim_paths.keys())

In [341]:
def format_runtime(runtime_minutes: float, 
                   nb_round: int = 2) -> str:
    """
    Simple function to format the runtime in minutes appropriately.

    Args:
        runtime_minutes (float): Runtime in minutes
        nb_round (int, optional): Number of digits for rounding. Defaults to 2.

    Returns:
        str: Nicely formatted runtime
    """
    runtime_string = str(np.round(runtime_minutes, nb_round))
    before, after = runtime_string.split(".")
    if len(after) == 1:
        runtime_string = before + "." + after + "0" 
        
    if len(runtime_string) == 4:
        runtime_string = "\\phantom{0}" + runtime_string
        
    return runtime_string

In [342]:
def fetch_runtime_injection(run_dir: str,
                      nb_round: int = 2) -> float:
    """
    Fetch the runtime of a given run directory of a jim run.

    Args:
        run_dir (str): Directory of the run, i.e. the outdir

    Returns:
        float: Runtime.
    """
    
    # Fetch runtimes of the individual parts:
    with open(run_dir + "/runtime.txt", "r") as f:
        runtime_seconds = float(f.read())
        
    runtime_minutes = runtime_seconds / 60
    runtime_string = format_runtime(runtime_minutes, nb_round)
            
    return runtime_seconds, runtime_string

In [343]:
def fetch_runtime_event(run_dir: str,
                      nb_round: int = 2) -> float:
    """
    Fetch the runtime of a given run directory of a jim run.

    Args:
        run_dir (str): Directory of the run, i.e. the outdir

    Returns:
        float: Runtime.
    """
    
    # Fetch runtimes of the individual parts:
    with open(run_dir + "/runtime.txt", "r") as f:
        runtime_seconds = float(f.read())
        
    # Note: the reruns were done with 1000 loops for evosax, but the fixed ref params used 2000 loops, so adding that once more
    with open(run_dir + "/runtime_evosax.txt", "r") as f:
        runtime_seconds += float(f.read())
        
    runtime_minutes = runtime_seconds / 60
    runtime_string = format_runtime(runtime_minutes, nb_round)
            
    return runtime_seconds, runtime_string

In [344]:
def fetch_runtime_evosax_event(run_dir: str,
                               nb_round: int = 2) -> float:
    """
    Fetch the runtime of a given real event run spent only on the evosax (computing reference parameters).

    Args:
        run_dir (str): Directory of the run, i.e. the outdir

    Returns:
        float: Runtime.
    """
    
    # Note: the reruns were done with 1000 loops for evosax, but the fixed ref params used 2000 loops, so adding that once more
    with open(run_dir + "/runtime_evosax.txt", "r") as f:
        runtime_seconds = 2 * float(f.read())
        
    runtime_minutes = runtime_seconds / 60
    runtime_string = format_runtime(runtime_minutes, nb_round)
            
    return runtime_seconds, runtime_string

In [345]:
def fetch_runtime_bilby(event_name: str, 
                        nb_round: int = 2,
                        convert_to_hours: bool = True) -> tuple[float, str]:
    """
    Fetch and format the runtime of analyzing one of the real events for bilby. 

    Args:
        event_name (str): Identifier of the event, format GW_WF, e.g. GW170817_TaylorF2
        outdir_name (str): Name of the outdir, e.g. `outdir`
        runtime_filename (str, optional): _description_. Defaults to "runtime.txt".
        nb_round (int, optional): _description_. Defaults to 2.

    Returns:
        tuple: Tuple of float value, being the runtime in seconds, and a string with the runtime in minutes
    """
    
    path = paths_dict[event_name]["bilby"]
    print(f"Bilby path: {path}")
    with open(path, "r") as f:
        bilby_data = json.load(f)
        runtime_seconds = float(bilby_data["sampling_time"])
        
    runtime_minutes = runtime_seconds / 60
    if convert_to_hours:
        runtime_minutes /= 60 # TODO fix the naming, or not, if I am too lazy in the end
    runtime_string = format_runtime(runtime_minutes, nb_round)
    
    return runtime_seconds, runtime_string

## Injections runtimes

In [346]:
runtimes_dict = {} 

for name in outdirs_injections.keys():
    outdir = outdirs_injections[name]
    runtimes = []
    for subdir in os.listdir(outdir):
        rundir = outdir + subdir
        runtime_path = rundir + "/runtime.txt"
        
        try:
            runtime, _ = fetch_runtime_injection(rundir)
            runtimes.append(runtime)
        except Exception as e:
            print(f"Skipping {rundir}, error: {e}")
            
    runtimes_dict[name] = np.array(runtimes)

Skipping ../injections/outdir_TF2/injection_54, error: [Errno 2] No such file or directory: '../injections/outdir_TF2/injection_54/runtime.txt'


Make a histogram of the runtimes, more as a sanity check for myself.

In [347]:
minutes = True # whether to convert runtimes to minutes rather than seconds

if minutes:
    denominator = 60
else:
    denominator = 1

hist_TF2, edges = np.histogram(runtimes_dict["TF2"] / denominator, bins = 20, density = True)
hist_NRTv2, _ = np.histogram(runtimes_dict["NRTv2"] / denominator, bins = edges, density = True)

if minutes:
    xlabel = "Runtime (min)"
else:
    xlabel = "Runtime (s)"

stairs_kwargs = {"linewidth": 2}

# Create the figure
plt.figure(figsize=(10, 6))
plt.stairs(hist_TF2, edges, label = "TF2", **stairs_kwargs)
plt.stairs(hist_NRTv2, edges, label = "NRTv2", **stairs_kwargs)

plt.legend()
plt.xlabel(xlabel)
plt.ylabel("Density")
# plt.show()
plt.close()

## Runtimes for compilation and initializing the likelihood

In [348]:
runtimes_compilation_filename = "../data/runtimes_compilation.txt"
runtimes_compilation = np.loadtxt(runtimes_compilation_filename, delimiter=",")

compilation_mean, compilation_std = np.mean(runtimes_compilation), np.std(runtimes_compilation)
print(f"Compilation: mean +/- std: {compilation_mean / 60:.2f} +/- {compilation_std / 60:.2f} minutes")

compilation_mean_minutes = compilation_mean / 60

Compilation: mean +/- std: 2.59 +/- 0.33 minutes


## Final numbers

Can subtract the time for compiling etc, if wanted:

In [349]:
jim_runtimes_injections_float = {} # in seconds
jim_runtimes_injections = {}
jim_runtimes_injections_median = {}
jim_runtimes_injections_median_str = {}
subtract_number = compilation_mean_minutes # in case we want to extract the compilation time for reporting in the paper
subtract_number = 0 # in case we DO NOT want to extract the compilation time for reporting in the paper

jim_total_runtime_injections = 0

for name, runtimes in runtimes_dict.items():
    
    # subtract compilation time in seconds
    jim_total_runtime_injections += np.sum(runtimes - subtract_number * 60)
    
    ### Mean +/- std:
    mean_str = format_runtime(np.mean(runtimes / denominator) - subtract_number)
    std_str = format_runtime(np.std(runtimes / denominator))
    median_value = np.median(runtimes / denominator) - subtract_number
    median_str = format_runtime(median_value)
    
    jim_runtimes_injections_float[name] = np.mean(runtimes) # in seconds
    jim_runtime_str = r"$({} \pm {})$ min".format(mean_str, std_str)
    print(f"Mean and std for {name}")
    print(jim_runtime_str)
    jim_runtimes_injections[name] = jim_runtime_str
    
    ### Median:
    median_str = r"${}$ min".format(median_str)
    
    jim_runtimes_injections_median[name] = median_value
    jim_runtimes_injections_median_str[name] = median_str
    
    # Print
    print(f"Median for {name}")
    print(median_value)
    print(median_str)

Mean and std for TF2
$(27.09 \pm 12.18)$ min
Median for TF2
24.758970030148824
$24.76$ min
Mean and std for NRTv2
$(21.10 \pm \phantom{0}9.16)$ min
Median for NRTv2
18.02008366982142
$18.02$ min


## Fetch jim runtimes

In [350]:
run_names = list(paths_dict.keys())
print(run_names)

jim_runtimes_float = {}
jim_runtimes_str = {}

jim_runtimes_float_evosax = {}
jim_runtimes_str_evosax = {}

jim_runtimes_float_sampling = {} # total runtime - evosax time
jim_runtimes_str_sampling = {}


for run in run_names:
    print(run)
    path = paths_dict[run]["jim"]
    path = path.replace("/outdir/", "/outdir_snellius/")
    print(f"Fetching runtimes from run: {path}")
    runtime_float, runtime_str = fetch_runtime_event(path)
    runtime_float_evosax, runtime_str_evosax = fetch_runtime_evosax_event(path)
    print(runtime_float)
    print(runtime_float_evosax)
    
    runtime_float_sampling = runtime_float - runtime_float_evosax
    runtime_str_sampling = format_runtime(runtime_float_sampling / 60)
    
    # Overall runtime
    jim_runtimes_float[run] = runtime_float
    jim_runtimes_str[run] = runtime_str
    
    # Runtime for evosax
    jim_runtimes_float_evosax[run] = runtime_float_evosax
    jim_runtimes_str_evosax[run] = runtime_str_evosax
    
    # Runtime for total - evosax
    jim_runtimes_float_sampling[run] = runtime_float_sampling
    jim_runtimes_str_sampling[run] = runtime_str_sampling

['GW170817_TaylorF2', 'GW170817_NRTidalv2', 'GW190425_TaylorF2', 'GW190425_NRTidalv2']
GW170817_TaylorF2
Fetching runtimes from run: /home/thibeau.wouters/TurboPE-BNS/real_events/GW170817_TaylorF2/outdir_snellius/
1602.3845567703247
582.1537246704102
GW170817_NRTidalv2
Fetching runtimes from run: /home/thibeau.wouters/TurboPE-BNS/real_events/GW170817_NRTidalv2/outdir_snellius/
2022.7856409549713
341.49449348449707
GW190425_TaylorF2
Fetching runtimes from run: /home/thibeau.wouters/TurboPE-BNS/real_events/GW190425_TaylorF2/outdir_snellius/
1296.861796617508
307.6143078804016
GW190425_NRTidalv2
Fetching runtimes from run: /home/thibeau.wouters/TurboPE-BNS/real_events/GW190425_NRTidalv2/outdir_snellius/
1291.1365008354187
368.90935134887695


## Fetch pbilby runtimes

These are the numbers that Peter gave me:

In [351]:
# # Override with the numbers obtained from the log file of the runs instead:
# bilby_runtimes_total = {}
# bilby_runtimes_total["GW170817_TaylorF2"]  = 0.0
# bilby_runtimes_total["GW170817_NRTidalv2"] = 12 * (60) ** 2 + 34 * 60 + 15
# bilby_runtimes_total["GW190425_TaylorF2"]  = 0.0
# bilby_runtimes_total["GW190425_NRTidalv2"] =  5 * (60) ** 2 + 39 * 60 + 38

In [352]:
bilby_runtimes_float = {}
bilby_runtimes_str = {}
for run in run_names:
    path = paths_dict[run]["bilby"]
    print(path)
    runtime_float, runtime_string = fetch_runtime_bilby(run, convert_to_hours=True)
        
    bilby_runtimes_float[run] = runtime_float
    bilby_runtimes_str[run] = runtime_string
    
bilby_runtimes_total = bilby_runtimes_float

/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW170817_TF2_with_tukey_fix_result.json
Bilby path: /home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW170817_TF2_with_tukey_fix_result.json


/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW170817_IMRPhenomD_NRTidalv2_result.json
Bilby path: /home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW170817_IMRPhenomD_NRTidalv2_result.json
/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW190425_TF2_with_tukey_fix_result.json
Bilby path: /home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW190425_TF2_with_tukey_fix_result.json
/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW190425_IMRPhenomD_NRTidalv2_result.json
Bilby path: /home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/GW190425_IMRPhenomD_NRTidalv2_result.json


In [353]:
# Add dollar signs and minute string
for key, value in jim_runtimes_str.items():
    jim_runtimes_str[key] = f"${value}$ min"

In [354]:
assert list(jim_runtimes_float.keys()) == list(bilby_runtimes_float.keys())
assert list(jim_runtimes_str.keys()) == list(bilby_runtimes_str.keys())

Put hours in the formatting for pbilby

In [355]:
for key, value in bilby_runtimes_str.items():
    bilby_runtimes_str[key] = f"${value}$ h"

## Get the ratio of runtimes

In [356]:
runtimes_ratio_float = {}
runtimes_ratio_str = {}

for key in jim_runtimes_float.keys():
    ratio = bilby_runtimes_float[key] / jim_runtimes_float[key]
    runtimes_ratio_float[key] = ratio
    print(ratio)
    runtimes_ratio_str[key] = f"${int(ratio)} \\times $"

21.655774466487056
19.551606753214244
11.334590495563338
13.081609338030004


## Combine into table

This is an older version, remove if not used anymore

In [357]:
# latex_code = '\\begin{{tabular}}{{l l c c c}}\n Event & Waveform & \\texttt{{jim}} & \\texttt{{pbilby}} & Speed-up \\\\\n \hline\\hline\n\\multirow{{2}}{{*}}{{GW170817}} & TF2 & {} & {} & {} \\\\\n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{GW190425}}  & TF2 & {} & {} & {} \\\\ \n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{Injection}} & TF2 & {} & -- & -- \\\\\n& NRTv2 & {} & -- & -- \\\\\n\\hline\\hline\n\\end{{tabular}}'\
# .format(  jim_runtimes_str["GW170817_TaylorF2"],
#         bilby_runtimes_str["GW170817_TaylorF2"],
#         runtimes_ratio_str["GW170817_TaylorF2"],
#           jim_runtimes_str["GW170817_NRTidalv2"],
#         bilby_runtimes_str["GW170817_NRTidalv2"], 
#         runtimes_ratio_str["GW170817_NRTidalv2"],
#           jim_runtimes_str["GW190425_TaylorF2"], 
#         bilby_runtimes_str["GW190425_TaylorF2"], 
#         runtimes_ratio_str["GW190425_TaylorF2"],
#           jim_runtimes_str["GW190425_NRTidalv2"], 
#         bilby_runtimes_str["GW190425_NRTidalv2"],
#         runtimes_ratio_str["GW190425_NRTidalv2"], 
#         jim_runtimes_injections_median_str["TF2"], 
#         jim_runtimes_injections_median_str["NRTv2"]
#        )

# print(latex_code)

## Environmental impact

- For the GPU consumption, see [1] or [2]: we have 400W for a A100 GPU installed in SXM, which is the case for Snellius [3]. 

- For the CPUs used, look at Ref. [4].

- Conversions to CO2 and trees: these are taken from Ref. [5]

-  In short: "For the capture of 1 tonne of CO2 emissions ~50 trees must grow for one year [6]
^using 0.905 kg of CO2 per kilowatt hour [7]."

- For NL, the most recent value is 0.328 kg CO2 per kWh [8].

[1] https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/a100/pdf/PB-10577-001_v02.pdf

[2] https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/a100/pdf/nvidia-a100-datasheet-nvidia-us-2188504-web.pdf 

[3] https://servicedesk.surf.nl/wiki/display/WIKI/Interactive+development+GPU+node 

[4] https://ark.intel.com/content/www/us/en/ark/products/136874/intel-xeon-platinum-8174-processor-33m-cache-3-10-ghz.html

[5] https://git.ligo.org/lscsoft/parallel_bilby/-/wikis/O4-Review#pbilby-201-202 

[6] https://www.climateneutralgroup.com/en/news/what-exactly-is-1-tonne-of-co2-v2/

[7] https://www.dcceew.gov.au/climate-change/publications/national-greenhouse-gas-inventory-quarterly-updates

[8] https://www.co2emissiefactoren.nl/lijst-emissiefactoren/

In [358]:
kwh_to_co2 = 0.328 # kg CO2 per kWh
co2_to_trees = (1 / 1000) * 50 # trees per kg CO2, growing for one year

consumption_cpu = 0.240 # kilowatts
consumption_gpu = 0.400 # kilowatts
# N_cores = 480
N_gpu = 1
N_runs = 204

Average runtime for jim:

In [359]:
total_runtime_events = np.sum(list(jim_runtimes_float.values()))
avg_runtime_jim = (jim_total_runtime_injections + total_runtime_events) / N_runs # first in seconds
print(avg_runtime_jim) # before: 1432.4360346029468


# in minutes
avg_runtime_jim /= 60
print(f"Average runtime jim (minutes):", avg_runtime_jim) # before: 23.873933910049114

# in hours
avg_runtime_jim /= 60
print(f"Average runtime jim (hours):", avg_runtime_jim) # before: 0.39789889850081855

1490.1688831448555
Average runtime jim (minutes): 24.83614805241426
Average runtime jim (hours): 0.413935800873571


Average runtime for bilby

In [360]:
avg_runtime_bilby = np.mean(list(bilby_runtimes_total.values())) / 3600
print("Average runtime bilby (hours):", avg_runtime_bilby)

Average runtime bilby (hours): 7.349939490833334


In [361]:
consumption_jim = {}
for key in jim_runtimes_float.keys():
    consumption_jim[key] = N_gpu * consumption_gpu * (jim_runtimes_float[key] / 3600)
    
consumption_jim["avg"] = N_gpu * consumption_gpu * avg_runtime_jim
    
for key, value in consumption_jim.items():
    print(f"Consumption for {key}: {value:.2f} kWh")

Consumption for GW170817_TaylorF2: 0.18 kWh
Consumption for GW170817_NRTidalv2: 0.22 kWh
Consumption for GW190425_TaylorF2: 0.14 kWh
Consumption for GW190425_NRTidalv2: 0.14 kWh
Consumption for avg: 0.17 kWh


In [362]:
N_cpu = 10

consumption_bilby = {}
for key in bilby_runtimes_float.keys():
    consumption_bilby[key] = N_cpu * consumption_cpu * (bilby_runtimes_float[key] / 3600)
    
consumption_bilby["avg"] = N_cpu * consumption_cpu * avg_runtime_bilby
    
for key, value in consumption_bilby.items():
    print(f"Consumption for {key}: {value:.2f} kWh")

Consumption for GW170817_TaylorF2: 23.13 kWh
Consumption for GW170817_NRTidalv2: 26.37 kWh
Consumption for GW190425_TaylorF2: 9.80 kWh
Consumption for GW190425_NRTidalv2: 11.26 kWh
Consumption for avg: 17.64 kWh


In [363]:
consumption_ratio = {}
consumption_ratio_str = {}
for key in consumption_jim.keys():
    this_ratio = int(np.round(consumption_bilby[key] / consumption_jim[key]))
    consumption_ratio[key] = this_ratio
    consumption_ratio_str[key] = r"${} \times$".format(this_ratio)
    
for key, value in consumption_ratio.items():
    print(f"Consumption for {key}: {value} x")

Consumption for GW170817_TaylorF2: 130 x
Consumption for GW170817_NRTidalv2: 117 x
Consumption for GW190425_TaylorF2: 68 x
Consumption for GW190425_NRTidalv2: 78 x
Consumption for avg: 107 x


Convert to trees now

In [364]:
def get_number_of_killed_trees(kwh: float) -> float:
    """
    Get the number of trees killed by the energy consumption of our runs in kWh.

    Args:
        kwh (float): Energy consumption in kWh

    Returns:
        float: Number of trees killed
    """
    return kwh * kwh_to_co2 * co2_to_trees

def get_number_of_tonne_co2(kwh: float) -> float:
    """
    Get the number of trees killed by the energy consumption of our runs in kWh.

    Args:
        kwh (float): Energy consumption in kWh

    Returns:
        float: Number of trees killed
    """
    return kwh * kwh_to_co2 / 1000

## Trees

In [365]:
trees_jim = {}
for key, value in consumption_jim.items():
    trees_jim[key] = get_number_of_killed_trees(value)
    
for key, value in trees_jim.items():
    print(f"Trees killed for {key}: {value:.5f}")
    
# Find the average:
avg_nb_killed_trees_jim = np.mean(list(trees_jim.values()))

Trees killed for GW170817_TaylorF2: 0.00292
Trees killed for GW170817_NRTidalv2: 0.00369
Trees killed for GW190425_TaylorF2: 0.00236
Trees killed for GW190425_NRTidalv2: 0.00235
Trees killed for avg: 0.00272


In [366]:
trees_bilby = {}
for key, value in consumption_bilby.items():
    trees_bilby[key] = get_number_of_killed_trees(value)
    
for key, value in trees_bilby.items():
    print(f"Trees killed for {key}: {value:.2f}")

Trees killed for GW170817_TaylorF2: 0.38
Trees killed for GW170817_NRTidalv2: 0.43
Trees killed for GW190425_TaylorF2: 0.16
Trees killed for GW190425_NRTidalv2: 0.18
Trees killed for avg: 0.29


For a comparison **about the whole paper**, just use this average and the number of runs done to allow for an easier comparison between the 2:

In [367]:
# avg_nb_killed_trees_jim = np.mean(list(trees_jim.values()))
# avg_nb_killed_trees_bilby = np.mean(list(trees_bilby.values()))

avg_nb_killed_trees_jim = trees_jim["avg"]
avg_nb_killed_trees_bilby = trees_bilby["avg"]

avg_nb_killed_trees_jim_paper = N_runs * avg_nb_killed_trees_jim
avg_nb_killed_trees_bilby_paper = N_runs * avg_nb_killed_trees_bilby

print(f"Average number of killed trees for writing the paper with   jim: {(avg_nb_killed_trees_jim_paper):.2f}")
print(f"Average number of killed trees for writing the paper with bilby: {(avg_nb_killed_trees_bilby_paper):.2f}")

Average number of killed trees for writing the paper with   jim: 0.55
Average number of killed trees for writing the paper with bilby: 59.02


## CO2

In [368]:
co2_jim = {}
for key, value in consumption_jim.items():
    co2_jim[key] = get_number_of_tonne_co2(value)
    
for key, value in co2_jim.items():
    print(f"Tonne CO2 for {key}: {value:.5f}")

Tonne CO2 for GW170817_TaylorF2: 0.00006
Tonne CO2 for GW170817_NRTidalv2: 0.00007
Tonne CO2 for GW190425_TaylorF2: 0.00005
Tonne CO2 for GW190425_NRTidalv2: 0.00005
Tonne CO2 for avg: 0.00005


In [369]:
co2_bilby = {}
for key, value in consumption_bilby.items():
    co2_bilby[key] = get_number_of_tonne_co2(value)
    
for key, value in co2_bilby.items():
    print(f"Tonne CO2 for {key}: {value:.5f}")

Tonne CO2 for GW170817_TaylorF2: 0.00759
Tonne CO2 for GW170817_NRTidalv2: 0.00865
Tonne CO2 for GW190425_TaylorF2: 0.00321
Tonne CO2 for GW190425_NRTidalv2: 0.00369
Tonne CO2 for avg: 0.00579


## Get environmental impact as a table

In [370]:
def my_format(number: float, nb_round: int = 2):
    my_string = str(np.round(number, nb_round))
    return my_string

\\def\\x{{1.5cm}} \n 

p{{\\x}}

In [409]:
latex_code = '& kWh & $\\rm{{CO}}_2$ [$10^3$ kg] & Trees${{}}^\dagger$ \\\\\n \hline\\hline\n \\textsc{{Jim}} & $\\phantom{{00}}{}$ & $\\phantom{{0}}{}$ & $\\phantom{{000}}{}$ \\\\\n \\textsc{{pBilby}} & ${}$ & ${}$ & ${}$ \\\\\n\\hline\\hline'\
.format(my_format(N_runs * consumption_jim["avg"]),
        my_format(N_runs * co2_jim["avg"]),
        my_format(avg_nb_killed_trees_jim_paper),
        my_format(N_runs * consumption_bilby["avg"]),
        my_format(N_runs * co2_bilby["avg"]),
        my_format(avg_nb_killed_trees_bilby_paper),
       )

print(latex_code)

& kWh & $\rm{CO}_2$ [$10^3$ kg] & Trees${}^\dagger$ \\
 \hline\hline
 \textsc{Jim} & $\phantom{00}33.78$ & $\phantom{0}0.01$ & $\phantom{000}0.55$ \\
 \textsc{pBilby} & $3598.53$ & $1.18$ & $59.02$ \\
\hline\hline


In [372]:
np.floor(avg_nb_killed_trees_bilby_paper / avg_nb_killed_trees_jim_paper)

106.0

## Update the table with energy saved rather than speedup

In [373]:
# latex_code = '\\begin{{tabular}}{{l l c c c}}\n Event & Waveform & \\textsc{{Jim}} & \\textsc{{pBilby}} & Energy saved \\\\\n \hline\\hline\n\\multirow{{2}}{{*}}{{GW170817}} & TF2 & {} & {} & {} \\\\\n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{GW190425}}  & TF2 & {} & {} & {} \\\\ \n & NRTv2 & {} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{Injection}} & TF2 & {} & -- & -- \\\\\n& NRTv2 & {} & -- & -- \\\\\n\\hline\\hline\n\\end{{tabular}}'\
# .format(  jim_runtimes_str["GW170817_TaylorF2"],
#         bilby_runtimes_str["GW170817_TaylorF2"],
#          consumption_ratio_str["GW170817_TaylorF2"],
#          jim_runtimes_str["GW170817_NRTidalv2"],
#         bilby_runtimes_str["GW170817_NRTidalv2"], 
#          consumption_ratio_str["GW170817_NRTidalv2"],
#            jim_runtimes_str["GW190425_TaylorF2"], 
#         bilby_runtimes_str["GW190425_TaylorF2"], 
#           consumption_ratio_str["GW190425_TaylorF2"],
#            jim_runtimes_str["GW190425_NRTidalv2"], 
#         bilby_runtimes_str["GW190425_NRTidalv2"],
#          consumption_ratio_str["GW190425_NRTidalv2"], 
#         jim_runtimes_injections["TF2"], 
#         jim_runtimes_injections["NRTv2"]
#        )

# print(latex_code)

## Without the final column

In [374]:
jim_runtimes_str_sampling

{'GW170817_TaylorF2': '17.00',
 'GW170817_NRTidalv2': '28.02',
 'GW190425_TaylorF2': '16.49',
 'GW190425_NRTidalv2': '15.37'}

In [406]:
def make_table_entry_str_event(key):
    return "$(" + jim_runtimes_str_evosax[key].replace("\phantom{0}", "") + " + " + jim_runtimes_str_sampling[key] + ")$ min"

def make_table_entry_str_injection(key):
    return "$\phantom{{(0.000 + }} {}\phantom{{)}}$ min".format(np.round(jim_runtimes_injections_median[key], 2))

In [407]:
jim_runtimes_injections_median["TF2"]
make_table_entry_str_injection("TF2")

'$\\phantom{(0.000 + } 24.76\\phantom{)}$ min'

\\footnotesize{{($480$ cores)}}

In [408]:

latex_code = '\\begin{{tabular}}{{l l c c}}\n Event & WF & \\textsc{{Jim}} \\footnotesize{{($1$ GPU)}} & \\textsc{{pBilby}} \\footnotesize{{($480$ cores)}} \\\\\n  \hline\\hline\n \\multirow{{2}}{{*}}{{GW170817}} & \\texttt{{TF2}} & {} & {} \\\\\n & \\texttt{{NRTv2}} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{GW190425}}  & \\texttt{{TF2}} & {} & {} \\\\ \n & \\texttt{{NRTv2}} & {} & {} \\\\ \hline\n\\multirow{{2}}{{*}}{{Injection}} & \\texttt{{TF2}} & {} & --  \\\\\n& \\texttt{{NRTv2}} & {} & --  \\\\\n\\hline\\hline\n\\end{{tabular}}'\
.format(make_table_entry_str_event("GW170817_TaylorF2"),
        bilby_runtimes_str["GW170817_TaylorF2"],
         make_table_entry_str_event("GW170817_NRTidalv2"),
        bilby_runtimes_str["GW170817_NRTidalv2"], 
         make_table_entry_str_event("GW190425_TaylorF2"), 
        bilby_runtimes_str["GW190425_TaylorF2"], 
         make_table_entry_str_event("GW190425_NRTidalv2"),
        bilby_runtimes_str["GW190425_NRTidalv2"],
        make_table_entry_str_injection("TF2"), 
        make_table_entry_str_injection("NRTv2")
       )

print(latex_code)

\begin{tabular}{l l c c}
 Event & WF & \textsc{Jim} \footnotesize{($1$ GPU)} & \textsc{pBilby} \footnotesize{($480$ cores)} \\
  \hline\hline
 \multirow{2}{*}{GW170817} & \texttt{TF2} & $(9.70 + 17.00)$ min & $\phantom{0}9.64$ h \\
 & \texttt{NRTv2} & $(5.69 + 28.02)$ min & $10.99$ h \\ \hline
\multirow{2}{*}{GW190425}  & \texttt{TF2} & $(5.13 + 16.49)$ min & $\phantom{0}4.08$ h \\ 
 & \texttt{NRTv2} & $(6.15 + 15.37)$ min & $\phantom{0}4.69$ h \\ \hline
\multirow{2}{*}{Injection} & \texttt{TF2} & $\phantom{(0.000 + } 24.76\phantom{)}$ min & --  \\
& \texttt{NRTv2} & $\phantom{(0.000 + } 18.02\phantom{)}$ min & --  \\
\hline\hline
\end{tabular}


## Some further timing info:

In [377]:
def report_timings_real_event_jim(path,
                                  nb_round: int = 2, 
                                  convert_to_minutes: bool = True):
    
    path = path["jim"]
    runtime_evosax =  2 * np.loadtxt(path + "/runtime_evosax.txt") # twice as many loops used to fetch the params than the rerun had
    runtime_training = np.loadtxt(path + "/runtime_training.txt")
    runtime_production = np.loadtxt(path + "/runtime_production.txt")
    total = np.loadtxt(path + "/runtime.txt")
    
    extra_string = "seconds"
    
    if convert_to_minutes:
        runtime_evosax /= 60
        runtime_training /= 60
        runtime_production /= 60
        total /= 60
        extra_string = "minutes"
        
    
    runtime_evosax = np.round(runtime_evosax, nb_round)
    runtime_training = np.round(runtime_training, nb_round)
    runtime_production = np.round(runtime_production, nb_round)
    total = np.round(total, nb_round)
    
    
    print(f"Runtime evosax: {runtime_evosax} {extra_string}")
    print(f"Runtime training: {runtime_training} {extra_string}")
    print(f"Runtime production: {runtime_production} {extra_string}")
    print(f"Total = {total} {extra_string}")

In [378]:
for name, path in paths_dict.items():
    print(" ==== event ====")
    print(name)
    report_timings_real_event_jim(path)

 ==== event ====
GW170817_TaylorF2
Runtime evosax: 9.7 minutes
Runtime training: 9.82 minutes
Runtime production: 0.19 minutes
Total = 21.86 minutes
 ==== event ====
GW170817_NRTidalv2
Runtime evosax: 5.69 minutes
Runtime training: 21.11 minutes
Runtime production: 0.85 minutes
Total = 30.87 minutes
 ==== event ====
GW190425_TaylorF2
Runtime evosax: 5.13 minutes
Runtime training: 10.98 minutes
Runtime production: 0.39 minutes
Total = 19.05 minutes
 ==== event ====
GW190425_NRTidalv2
Runtime evosax: 6.15 minutes
Runtime training: 11.59 minutes
Runtime production: 0.48 minutes
Total = 18.44 minutes
